# Orchestrator-Workers Workflow

In this notebook, we'll explore the orchestrator-workers pattern — a powerful workflow design for creating flexible AI systems that can tackle complex, unpredictable tasks.
Using Kotlin and Claude via LangChain4j,
we'll implement a practical example that demonstrates
how a main _"orchestrator"_ LLM can coordinate multiple _"worker"_ LLMs to achieve better results.

## What is the orchestrator-workers pattern?

The orchestrator-workers pattern involves using a central LLM (the orchestrator) to analyze a complex task,
break it down dynamically into subtasks, and then delegate these subtasks to specialized worker LLMs.
Finally, the orchestrator synthesizes the results into a cohesive output.

![Orchestrator-Workers Workflow Diagram](image/orchestrator_workers.svg)

This workflow differs from simple parallelization because:
- The subtasks aren't predetermined but are identified dynamically by the orchestrator
- The orchestrator can make decisions about task allocation based on the specific input
- There's an explicit coordination layer that manages the overall process

### When to use this pattern

- Software development tasks that involve changes across multiple files
- Complex research or analysis requiring multiple perspectives
- Content creation with varying style requirements
- Multistep problem-solving with interdependent components

## Setting up environment

Let's start by configuring the Kotlin notebook with the necessary dependencies:

In [1]:
%useLatestDescriptors
%use langchain4j(model = anthropic)

## Defining data models

We'll need some data structures to represent tasks and responses:

In [2]:
import com.fasterxml.jackson.annotation.JsonCreator
import com.fasterxml.jackson.annotation.JsonProperty

/**
 * Represents a subtask assigned by the orchestrator that needs to be executed by the performer.
 */
data class Task @JsonCreator constructor(
    @JsonProperty("type") val type: String,
    @JsonProperty("description") val description: String
)

/**
 * The orchestra's response, containing a task analysis and breakdown into subtasks.
 */
data class OrchestratorResponse @JsonCreator constructor(
    @JsonProperty("analysis") val analysis: String,
    @JsonProperty("tasks") val tasks: List<Task>
)

data class WorkerResponse(val type: String, val description: String, val result: String) {
    override fun toString(): String {
        return """
            {
              "type": "$type",
              "description": "$description",
              "result": "$result"
            }
        """.trimIndent()
    }
}

These classes serve as the communication structures between our orchestrator and worker LLMs.

## Creating LLM Service

Next, define a simple interface for our LLM interactions:

In [3]:
interface LlmService {
    fun orchestrate(input: String): OrchestratorResponse
    fun perform(input: String): String
}

## Implementing the orchestrator

Now for the heart of our implementation — the `Orchestrator` class

In [4]:
import kotlinx.serialization.Serializable

/**
 * Break down tasks and run them in parallel using worker LLMs
 */
class Orchestrator(val llm: LlmService) {

    companion object {
        val orchestratorPrompt: (String) -> String = {
            """
            Analyze this task and break it down into 2-3 distinct approaches:

            Task: $it

            Return your response in this JSON format:
            {
              "analysis": "Explain your understanding of the task and which variations would be valuable. Focus on how each approach serves different aspects of the task.",
              "tasks": [
                {
                  "type": "formal",
                  "description": "Write a precise, technical version that emphasizes specifications"
                },
                {
                  "type": "conversational",
                  "description": "Write an engaging, friendly version that connects with readers"
                }
              ]
            }
            """.trimIndent()
        }

        @Serializable
        data class Task(val type: String, val description: String)

        val workerPrompt: (String, String, String) -> String =
            { originalTask, taskType, taskDescription ->
                """
                Generate content based on:
                Task: $originalTask
                Style: $taskType
                Guidelines: $taskDescription
                """.trimIndent()
            }
    }

    /**
     * Process task by brealing it down and running subtasks in parallel.
     */
    fun process(taskInput: String): Pair<String, List<WorkerResponse>> {
        val orchestratorInput = orchestratorPrompt(taskInput)
        val orchestratorResponse = llm.orchestrate(orchestratorInput)

        val (analysis, tasks) = orchestratorResponse

        println(
            """
            === ORCHESTRATOR OUTPUT ===
            ANALYSIS: ${orchestratorResponse.analysis}

            TASKS: ${orchestratorResponse.tasks}
            """.trimIndent()
        )

        val workerResults = tasks.map { task ->
            val workerInput = workerPrompt(taskInput, task.type, task.description)
            val workerResponse = llm.perform(workerInput)


            println(
                """
                === WORKER RESULT (${task.type}) ===
                $workerResponse
                """.trimIndent()
            )

            WorkerResponse(task.type, task.description, workerResponse)
        }

        return analysis to workerResults
    }
}

## Creating LLM Model

Let's set up the Claude model:

In [5]:
val model = AnthropicChatModel.builder()
    .apiKey(System.getenv("ANTHROPIC_API_KEY"))
    .modelName(AnthropicChatModelName.CLAUDE_3_7_SONNET_20250219)
    .maxTokens(4096)
    .temperature(0.1)
    .build()

## Running the workflow

Finally,create the llm service and process a task

In [6]:
val service = AiServices.create(LlmService::class.java, model)

val orchestrator = Orchestrator(service)
val result = orchestrator.process("Write a product description for a new eco-friendly water bottle")

=== ORCHESTRATOR OUTPUT ===
ANALYSIS: This task requires creating a product description for an eco-friendly water bottle. The key challenge is to effectively communicate both the functional benefits (durability, capacity, materials) and emotional benefits (environmental impact, lifestyle alignment) of the product. Different approaches would be valuable to address various customer segments and marketing contexts. Some customers may prioritize technical specifications and environmental credentials, while others might be more motivated by lifestyle benefits and emotional appeals.

TASKS: [Task(type=technical-environmental, description=Create a specification-focused description highlighting the bottle's eco-friendly materials, manufacturing process, and technical features with precise measurements and environmental impact data), Task(type=lifestyle-benefits, description=Develop a benefits-oriented description emphasizing how the bottle enhances the user's daily life while supporting their 

## How it works

1. The orchestrator LLM receives the initial task and analyzes it to identify distinct approaches (like formal and conversational styles)
2. Based on its analysis, the orchestrator dynamically generates subtasks with specific requirements
3. Each subtask is delegated to a worker LLM, which executes it with the focused parameters
4. The orchestrator collects the results from all workers

## Conclusion

The orchestrator-workers pattern offers a powerful approach to building AI systems that can tackle complex, unpredictable tasks.
By dynamically breaking down problems and delegating subtasks to specialized workers,
we can create more flexible, adaptable solutions.

Kotlin's expressive syntax and strong typing make it an excellent fit for implementing this pattern,
providing both readability and reliability.

This approach is particularly valuable when dealing with tasks where the exact steps aren't known in advance,
allowing our AI systems to adapt their strategies based on the specific needs of each input.